<a href="https://colab.research.google.com/github/SoumadeepMandal/Smart_chatbot/blob/main/chatBotTesti_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install newspaper3k lxml[html_clean]

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 53.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 5.8 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=71c4ce976c5097a641bb4598660135e881077d97bba32f583af57046f3c20ad1
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=1820034c90f8ccfb4bd1a779f62a11db7990b737db9bb48e9fb2e99d15c972a4
  Stored in directory: /root/.cache/pip/wheels/80/d5/72/9cd9eccc819636436c6a6e59c22a0fb1ec1

In [ ]:
import random
import nltk
import numpy as np
from newspaper import Article
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings


In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
varOcg = "chronic kidney disease expert assistant"

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
# Download the punkt_tab resource which might be required by newspaper's nlp()
nltk.download('punkt_tab')


from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def normalize(text):
    tokens = nltk.word_tokenize(text.lower())
    return ' '.join([lemmatizer.lemmatize(token) for token in tokens])

url = 'https://en.wikipedia.org/wiki/Kidney_disease'
article = Article(url)
article.download()
article.parse()
article.nlp()
corpus = article.text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
sentence_list = nltk.sent_tokenize(corpus)

In [ ]:
user_greetings = ['hi', 'hello', 'hey', 'whatsup', 'howdy', 'hola']
bot_greetings = ['Hello!', 'Hi there!', 'Hey!', 'Howdy!', 'Greetings!']

def greeting_response(text):
    for word in text.lower().split():
        if word in user_greetings:
            return random.choice(bot_greetings)
    return None

In [ ]:
exit_list = ['exit', 'bye', 'see you', 'break', 'quit']

In [ ]:
fallbacks = [
    "I'm not sure I understand. Could you rephrase?",
    "Sorry, I didn't catch that. Ask something else about kidney disease.",
    "I can help with symptoms, causes, or treatment. Try asking something specific."
]

In [ ]:
def bot_response(user_input):
    user_input = normalize(user_input)
    sentence_list.append(user_input)

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentence_list)
    similarity_scores = cosine_similarity(tfidf_matrix[-1], tfidf_matrix)
    similarity_scores_list = similarity_scores.flatten()
    index_scores = similarity_scores_list.argsort()[::-1]

    response = ''
    response_found = False
    added = 0

    for index in index_scores:
        if index != len(sentence_list) - 1 and similarity_scores_list[index] > 0:
            response += sentence_list[index] + ' '
            response_found = True
            added += 1
        if added >= 2:
            break

    sentence_list.pop()

    if not response_found:
        return random.choice(fallbacks)
    return response.strip()


In [ ]:
print("Doc 🤖: Hi! I'm your health assistant. Ask me anything about chronic kidney disease. Type 'exit' to leave.")

while True:
    user_input = input("You: ")
    if any(exit_word in user_input.lower() for exit_word in exit_list):
        print("Doc 🤖: Bye! Take care.")
        break
    elif greeting_response(user_input):
        print("Doc 🤖:", greeting_response(user_input))
    else:
        print("Doc 🤖:", bot_response(user_input))

Doc 🤖: Hi! I'm your health assistant. Ask me anything about chronic kidney disease. Type 'exit' to leave.
You: hi
Doc 🤖: Hi there!
You: what is Kidney disease?
Doc 🤖: Nephrosis is non-inflammatory kidney disease. COVID-19 is associated with kidney disease.
You: be more secific  
Doc 🤖: [11] This may be because sources of animal protein, animal fat, and cholesterol, and sweets are more acid-producing, while fruits, vegetables, legumes, and whole grains are more base-producing. Chronic kidney disease is defined as prolonged kidney abnormalities (functional and/or structural in nature) that last for more than three months.
You: what are the risk?
Doc 🤖: Currently, the underlying mechanisms are unclear. Polycystic kidneys are susceptible to infections and cancer.
You: what are the factors of this ?
Doc 🤖: "[8] The same article also goes on to say that millions of Americans are affected by this disease and it is very common. [11] This may be because sources of animal protein, animal fat, an